# Caching and uncaching with Spark

This is not meant to be a full tutorial, but rather a brief presentation on how to cache datasets with Krapsh. A full Spark tutorial can be found there: http://spark.apache.org/docs/latest/quick-start.html#caching

## Normal caching

In [16]:
:load KrapshDisplays
:extension OverloadedStrings
import KrapshDisplays(showGraph)
import KrapshDagDisplay(computeGraphToDisplayGraph)

In [17]:
import Spark.Core.Dataset
import Spark.Core.Context
import Spark.Core.Functions
import Spark.Core.Column
import Spark.Core.ColumnFunctions


conf = defaultConf {
        confEndPoint = "http://localhost",
        confRequestedSessionName = "session03_introduction" }

createSparkSessionDef conf

Error {ePath = NPath(), eMessage = "A default context already exist. If you wish to modify the exsting context, you must use modifySparkConfDef"}

In [18]:
-- Some internal utilities to show what is happening under the scene.
import Spark.Core.Internal.ContextStructures
import Spark.Core.Internal.ContextInternal
import Spark.Core.Internal.Utilities(forceRight)

Caching is a fundamental step when working repeatedly with large datasets. It is hard to get right, however, and it is a frequent source of performance issues for budding Spark users. Krapsh embeds some tools to help you diagnose caching issues.

Consider the following trivial program that creates a dataset, and runs a few computations on it (it computes the number of elements as well as their sums, and computes the mean out of it). Of course, Spark includes a mean operator so this would not be recommended in practice, but it is instructive to study caching issues.

In [19]:
ds = dataset ([1,2,3] :: [Int]) @@ "data"
-- Turns the dataset into a column and computes the sum of all the elements in this column
s1 = sumCol (asCol ds) @@ "sum"
-- Counts the element in the dataset
s2 = count ds @@ "count"
x = (s1 + s2) @@ "result"

Let us see what the computation graph looks like:

In [20]:
showGraph x

'>

For large datasets, this would be inefficient, because it would cause the original dataset `data` to be recomputed twice, to perform the `sum` and `count` operations.

We can instruct Spark to _cache_ the dataset (using the `cache` operator). But this will consume some precious memory, so we need to tell Spark to _uncache it_ once we are sure we are done. The operator to do that, unsurprisingly, is called `uncache`. Here is a first attempt that is not going to work:

In [21]:
-- cache
cachedData = cache ds @@ "cache"
s1 = sumCol (asCol cachedData) @@ "sum"
s2 = count cachedData @@ "count"
uncachedData = uncache cachedData @@ "uncache"
x = (s1 + s2) @@ "result"

Let us look at the graph of computations: the uncaching operation is missing!

In [22]:
showGraph x

'>

Because of laziness, the result must depend on the uncaching operation. Otherwise it is skipped when calculating the operations that need to be done.

Here is a second attempt, in which we explicitly add a _logical dependency_ between the uncaching and the result: the uncaching will happen before we compute the result. Adding a logical dependency is expressed with the `depends` function.

In [23]:
:t depends

depends :: forall loc a. ComputeNode loc a -> [UntypedNode] -> ComputeNode loc a

In [24]:
x = ((s1 + s2) `depends` [untyped uncachedData]) @@ "result"

Now we see the uncaching operation in the graph of operations:

In [25]:
showGraph x

'>

However, when we attempt to run it, it fails!

In [26]:
exec1Def x

Error {ePath = NPath(), eMessage = "Found some caching errors: [CachingFailure {cachingNode = cccd6.., uncachingNode = cb333.., escapingNode = 50f04..},CachingFailure {cachingNode = cccd6.., uncachingNode = cb333.., escapingNode = 6bc97..},CachingFailure {cachingNode = cccd6.., uncachingNode = cb333.., escapingNode = 7c3fa..}]"}

The message gives us a hint, telling us that it found some operations that are illegal.

When you look at the graph above, nothing indicates to Spark if it should first cache the data, or compute the sum or count operation. This ambiguity is in fact visible in the graph because the three nodes `sum`, `count` and `uncache` are represented on the _same line_. Which one should be done first?

Because of the lazy nature in Spark, it is a common mistake to uncache data while it is still going to be used later. Krapsh analyzes the graph of computations, and will refuse to run operations if they could be misintepreted by Spark. In that case, Krapsh can see the `sum` or `count` operation could happen _after_ we uncache the data, so it will ask us to resolve this ambiguity.

How do we fix it? Simply by ensuring that `sum` or `count` is run _before_ uncaching the data, using logical dependencies again:

In [27]:
uncachedData = ((uncache cachedData) `depends` [untyped s1, untyped s2]) @@ "uncache"
x = ((s1 + s2) `depends` [untyped uncachedData]) @@ "result"

showGraph x

'>

When trying to run this program now, Spark can properly launch the computations.

In [28]:
exec1Def x

[Debug] executeCommand1: computing observable result@org.spark.LocalPlus!int @(<unknown>:<unknown> <unknown>:0:0)
[Info] Sending computations at url: http://localhost:8081/computations/session03_introduction/1/create @(<unknown>:<unknown> <unknown>:0:0)
[Debug] executeCommand1: Tracked nodes are [(6bc97..,NPath(sum),int,sum),(7c3fa..,NPath(count),int,count),(82bf2..,NPath(result),int,result)] @(<unknown>:<unknown> <unknown>:0:0)
9

## Autocache

This business of caching and uncaching is clearly tedious and at the same time fairly mechanical. Can we automate all that? Yes! Krapsh includes an _autocache_ operator that acts as a hint: it instructs Spark to consider a specific dataset for potential caching. If Krapsh decides it is useful to cache this dataset, it will automatically insert caching and uncaching instructions.

Here is our original program, but this time we automate the caching process: no uncaching, and caching is done using the `autocache` operator.

In [29]:
cachedData = autocache ds @@ "autocache"
s1 = sumCol (asCol cachedData) @@ "sum"
s2 = count cachedData @@ "count"
x = (s1 + s2) @@ "result"

This is all that is required. For the sake of demonstration, let us see under the hood what transforms have been done by looking at the graph of computations. This is not required by casual users, only to understand the computations that are going to take place:

In [31]:
cg = forceRight (buildComputationGraph x)

In [32]:
performGraphTransforms 

performGraphTransforms :: SparkSession -> ComputeGraph -> Try ComputeGraph

Failed to load interface for `Spark.Core.Internal.ContextInteractive'
it is a hidden module in the package `krapsh-0.1.9.0'
Use -v to see a list of the files searched for.

In [30]:
cg = forceRight (buildComputationGraph x)
cg' = forceRight (performGraphTransforms cg)
computeGraphToDisplayGraph cg'

Couldn't match expected type `Either () b' with actual type `ComputeGraph -> Spark.Core.Try.Try ComputeGraph'
Relevant bindings include cg' :: b (bound at <interactive>:2:1)
Probable cause: `performGraphTransforms' is applied to too few arguments
In the first argument of `forceRight', namely `(performGraphTransforms interactive:Ghci234.cg)'
In the expression: forceRight (performGraphTransforms interactive:Ghci234.cg)

Couldn't match type `Spark.Core.Internal.ComputeDag.ComputeDag UntypedNode Spark.Core.Internal.DatasetStructures.StructureEdge' with `SparkSession'
Expected type: SparkSession
  Actual type: ComputeGraph
In the first argument of `performGraphTransforms', namely `interactive:Ghci234.cg'
In the first argument of `forceRight', namely `(performGraphTransforms interactive:Ghci234.cg)'

This is very close to the transforms that we have done manually! Krapsh has added extra logical dependencies to ensure that operations are done in the correct order.

To conclude:
 - use `autocache` when possible
 - if you need more fine-grained control over the placement and the order of operations, you can manually cache datasets using `cache` and `uncache`. Krapsh will prevent you from doing mistakes however.